# What this does

## Goal
- We want to find vocabularies that is commonly used in fakenews newspapers 

## How
1. Create a list of bigrams
2. Count their occurrences in the mainstream articles dataset
3. Count their occurrences in the fakenews articles dataset
4. Compare the differences of appreance of bigrams in mainstream articles and fakenews articles

## Results
- Identified bigrams commonly used in fakenews articles and not used in mainstream articles.
- A list of bigrams commonly used in fakenees with their frequency.

In [31]:
import sys
sys.path.append('../src/analyse')
from counter import Counter
from collections import Counter as CollCount
import nltk
from tqdm import tqdm

In [32]:
import numpy as np
counters = np.load('../data/articles.npy', allow_pickle=True)

In [33]:
def find(counter, words, bigrams, min_appearences=0, max_appearances=9223372036854775807):
    if words:
        fdist = counter.get_words()
    elif bigrams:
        fdist = counter.get_bigrams()
    else:
        return
    keys = set()
    for k,v in fdist.items():
        if min_appearences < v < max_appearances:
            keys.add(k)
    return keys , fdist

def shared(counter_a, counter_b, words, bigrams, appearences):
    """ outputs a set of words that appear at least t times in both a and b"""
    keys_a, fdist = find(counter_a, words, bigrams, appearences)
    keys_b, fdist = find(counter_b,words, bigrams, appearences)
    return keys_a & keys_b

def shared_multiple_sets(counters, words, bigrams, nr_sites, real, min_appearences=0, max_appearances=1000):
    """ Outputs words that appear at least t times and at least in x sets with the same label for real"""
    all = set()
    all_sets = []
    final_set = set()
    count_freq = nltk.FreqDist()
    for counter in tqdm(counters):
        # Do note, real here is a boolean, so can be false, it checks if "real" are the same
        if counter.real==real:
            new_set,new_freq = find(counter, words, bigrams, min_appearences=min_appearences, max_appearances=max_appearances)
            all_sets.append(new_set)
            all = all|new_set
            count_freq = count_freq+new_freq
    for word in all:
        count = 0
        for i in all_sets:
            if word in i:
                count+=1
        if count>=nr_sites:
            final_set.add(word)
    return final_set, count_freq


def difference(counters, words, bigrams, nr_fake_sites,nr_real_sites,fake_appearences, real_appearances):
    """ Outputs words which appear in at least nr_fakes_sites fake sites at least min_appearances times, but not in nr_real_sites real (below threshold)"""
    frequent_fake_words, fake_count_freq = shared_multiple_sets(counters,words, bigrams, nr_fake_sites,False,min_appearences=fake_appearences,max_appearances=999999)
    frequent_real_words, real_count_freq = shared_multiple_sets(counters,words, bigrams, nr_real_sites,True, min_appearences=real_appearances,max_appearances=999999)
    return frequent_fake_words - frequent_real_words, fake_count_freq, real_count_freq

In [38]:
""" Below prints differences for words"""
def different_words(counters, nr_fake_sites=1, nr_real_sites=1,fake_appearences=100, real_appearances=1):
    words, fake_freq, real_freq = difference(counters, words = True, bigrams=False,nr_fake_sites = nr_fake_sites, nr_real_sites=nr_real_sites,fake_appearences=fake_appearences, real_appearances=real_appearances)
    total_fake = sum(fake_freq.values())
    total_real = sum(real_freq.values())
    print(total_fake,", ", total_real)
    for word in words:
        # print(word, ", fake count:", fake_freq.get(word), ",real count:", real_freq.get(word))
        print(word, ",", fake_freq.get(word), ",", real_freq.get(word))

def different_bigrams(counters, nr_fake_sites=1, nr_real_sites=1,fake_appearences=100, real_appearances=1):
    bigrams, fake_freq, real_freq = difference(counters, words = False, bigrams=True,nr_fake_sites = nr_fake_sites, nr_real_sites=nr_real_sites,fake_appearences=fake_appearences, real_appearances=real_appearances)
    for bi in bigrams:
        # print(bi, ", fake count:", fake_freq.get(bi), ",real count:", real_freq.get(bi))
        print(bi, ",", fake_freq.get(bi), ",", real_freq.get(bi))

def different_bigrams_weight(counters):
    """ This function outputs the bigrams and their weight over all bigrams"""
    bigrams, fake_freq, real_freq = difference(counters, words = False, bigrams=True,nr_fake_sites = 1, nr_real_sites=1,fake_appearences=0, real_appearances=10)
    total_fake = sum(fake_freq.values())
    total_real = sum(real_freq.values())
    print(total_fake,", ", total_real)
    for bi in bigrams:
        # prints bigram, frequency in fake, weight of total, frequeuncy of real, weight of total bigrams
        print(bi, ",", fake_freq.get(bi), ",", real_freq.get(bi))

In [35]:
different_words(counters, nr_fake_sites=1, nr_real_sites=1,fake_appearences=100, real_appearances=1)

100%|██████████| 12/12 [00:10<00:00,  1.20it/s]

1082883 ,  4765824
chemtrails , 275 , 2
copypasten , 125 , 2
vaccinatieschade , 208 , 2
vaccinvrij , 917 , 1
geopendmeerklik , 474 , None
hpvvaccin , 151 , 1
pratenshare , 375 , None
geloofsforum , 800 , None
inplakken , 232 , 1
pixabay , 151 , None
redacteuropinieblogger , 181 , None
statines , 116 , None
tekstje , 125 , 1
transitiewebmastodonsocial , 168 , None
measles , 204 , None
pocket , 474 , None
siteregelsgeloofsdiscussies , 400 , None
galien , 174 , None
posting , 181 , None
aanklikken , 126 , None
frankema , 238 , None
engelbart , 617 , None
globalisten , 133 , 1
invulscherm , 108 , None
familieklik , 202 , None
intypen , 108 , 1
reijmerink , 127 , None
cursor , 108 , None
checker , 125 , 1
postings , 356 , None
steffers , 258 , None
checkers , 162 , None
mastodonsocial , 168 , None


In [36]:
different_bigrams(counters, nr_fake_sites=1, nr_real_sites=1,fake_appearences=25, real_appearances=50)

100%|██████████| 12/12 [00:24<00:00,  2.02s/it]


('mensen', 'lezen') , 205 , 4
('nederland', 'feminisme') , 42 , None
('zelfs', 'statistieken') , 32 , None
('ooit', 'virtuele') , 54 , None
('politiek', 'correcte') , 32 , 3
('opmars', 'nieuwe') , 41 , 1
('staatsgreep', 'duitsland') , 35 , None
('nooit', 'werken') , 32 , None
('90000', 'slachtoffers2601') , 34 , None
('verplichte', 'corona') , 75 , None
('genezen', 'hydroxychloroquine') , 49 , None
('yuval', 'noah') , 34 , 4
('decentrale', 'sociale') , 347 , None
('jouw', 'reactie') , 110 , None
('belangrijk', 'mensen') , 212 , 68
('server', 'mastodonsocial') , 168 , None
('bijwerkingen', 'vaccins') , 35 , 2
('besmettingen', 'doden') , 37 , 129
('staan', 'fact') , 62 , None
('onbeperkt', 'sluiten') , 51 , None
('document', 'chinese') , 46 , None
('handig', 'maak') , 55 , None
('corona', 'gevonden') , 35 , 4
('stijgt', 'ruim') , 27 , 1
('denk', 'denk') , 29 , 3
('gates', 'geeft') , 58 , 1
('media', 'berichten1003') , 28 , None
('nieuw', 'emailen') , 202 , None
('vernietigen', 'herstarte

('afstand', 'gecontroleerde') , 26 , None
('artsen', 'wetenschappers') , 197 , 10
('eisen', 'terugkeer') , 32 , None
('kunt', 'reactie') , 110 , None
('vanaf', 'september') , 40 , 87
('boek', 'vaccinvrij') , 114 , None
('straks', 'brits') , 31 , None
('corona', 'rubriek') , 137 , 1
('voorspelt', 'dagen') , 36 , None
('spaanse', 'griep2501') , 35 , None
('chemische', 'stoffen') , 40 , 3
('reden', 'waarom') , 87 , 71
('forever', 'anderhalve') , 36 , None
('forum', 'lees') , 400 , None
('zwangere', 'vrouwen') , 132 , 50
('bericht', 'handig') , 54 , None
('miljoen', 'gestorven2502') , 31 , None
('corona2903', 'chinese') , 26 , None
('vaccins', 'vaccinvrij') , 158 , None
('vriend', 'nieuw') , 202 , None
('lezers', 'grote') , 168 , None
('studie', 'lockdowns') , 29 , None
('rivm', 'column') , 28 , None
('coronavirus', 'inplantingen') , 32 , None
('stoppen2801', 'nieuwe') , 31 , None
('lichaam', 'betalingssysteem1304') , 33 , None
('gebruiken', 'oprichting') , 94 , None
('europa', 'grootste')

('verspreidt', 'informatie') , 32 , None
('gelekt', 'document') , 33 , None
('viroloog', 'waarschuwt') , 42 , 1
('artikelen', 'afgelopen') , 131 , None
('column', 'pierre') , 28 , None
('digitale', 'munt') , 31 , 1
('zero', 'hedge2') , 81 , None
('steeds', 'censuur') , 168 , None
('posted', 'beheer') , 31 , None
('vaccin', 'mensen') , 41 , 22
('2020060606', 'koppelt') , 60 , None
('vaccin', 'corona') , 41 , 52
('nieuwe', 'berichten') , 109 , 1
('ervoor', 'kiezen') , 44 , 41
('opgroeien', 'initiatiefneemster') , 144 , None
('leven0704', 'bill') , 30 , None
('internet', 'gebruikt') , 55 , None
('besluit', 'korte') , 55 , None
('deel', 'bericht') , 204 , None
('serieus', 'verplichte') , 59 , None
('pratenshare', 'this') , 375 , None
('tijdens', 'zwangerschap') , 49 , 13
('computerscherm', 'post') , 55 , None
('pocket', 'nieuw') , 474 , None
('vaccins', 'autisme') , 26 , None
('tekstje', 'tekst') , 108 , None
('zeer', 'grote') , 119 , 23
('noemt', 'coronavirus') , 36 , 4
('brother', 'tusse

In [39]:
different_bigrams_weight(counters)

100%|██████████| 12/12 [00:24<00:00,  2.08s/it]


1082875 ,  4765820
('compassie', 'betrokkenheid') , 1 , None
('virus', 'gepresenteerde') , 1 , None
('lockdown', 'fase') , 2 , None
('53jarige', 'rijswijk') , 1 , None
('medewerkers', 'farmaceutische') , 1 , None
('toetreding', 'albani') , 1 , None
('data', 'viruswaarheid') , 1 , None
('bijwerkingen', 'zelfs') , 2 , 2
('kort', 'ingaan') , 1 , 1
('materiaal', 'mensen') , 2 , 2
('httpswwwadnlpolitiekterugnaarnormaalisnogverwegzegtrutteafabe0cf', 'httpswwwhartvannederlandnlnieuws2020ruttecoronamaatregelennietslechtsbeperktversoepelen') , 1 , None
('vaccine', 'httpsthewashingtonstandardcomaustralianpmlifewontreturntonormaluntiltheresavaccine') , 1 , None
('bijna', 'vierde') , 1 , 1
('onafhankelijk', 'wetenschappelijk') , 20 , 1
('verwees', 'eigen') , 2 , 1
('enkel', 'gedaagde') , 1 , None
('bvaccin', 'rechtbank') , 1 , None
('blijkt', 'minstens') , 1 , 1
('liquid', 'manure') , 1 , None
('besproken', 'meestal') , 1 , None
('voordat', 'bepaalde') , 1 , 1
('meest', 'voorkomend') , 1 , None
('

('ruttekoningrivm', 'opgelost') , 1 , None
('mazelenvirus', 'jammer') , 1 , None
('approximately', '40000') , 1 , None
('niveau', 'gezond') , 1 , None
('drosten', 'jaar') , 1 , None
('tunnels', 'still') , 1 , None
('electrosmog', 'jaren') , 1 , None
('maatregelen', 'spoedwet') , 5 , None
('vanwege', 'verkeerde') , 1 , None
('moraal', 'weggehaald') , 1 , None
('biologische', 'nietgmovoedingsmiddelen') , 1 , None
('gezondheid', 'biedt') , 1 , None
('hoeft', 'daarna') , 1 , None
('tegelijkertijd', 'afgenomen') , 1 , None
('grofweg', 'dezelfde') , 1 , 4
('this', 'articledagelijks') , 1 , None
('aantal', 'meningitisgevallen') , 1 , None
('nepotisme', 'bevraagt') , 1 , None
('reservoir', 'ramp') , 1 , None
('onderzoekers', 'bezighielden') , 1 , None
('houden', 'buiten') , 3 , 9
('deuren', 'ander') , 1 , None
('beschrijft', 'tabaksontmoediging') , 1 , None
('artikel', 'stoornis') , 1 , None
('difterie', 'bacillen') , 2 , None
('justitie', 'tweede') , 1 , None
('wetenschappers', 'studie') , 2 

('artikel', 'april') , 5 , 1
('pediatrics', 'conferenties') , 1 , None
('vruchten', 'gelijke') , 1 , None
('langer', 'afgehaakt') , 1 , None
('ongeregeldheden', 'voordeden') , 1 , None
('kunstmatige', 'wolken') , 3 , None
('krijg', 'waalsdorpervlaktegevoel') , 1 , None
('vloer', 'europese') , 1 , None
('nodig', 'gearresteerd') , 1 , None
('postnl', 'gestopt') , 1 , None
('beter', 'recht') , 2 , 1
('levensbloed', 'werk') , 1 , None
('gezondheid', 'ergo') , 1 , None
('uitkomt', 'procedures') , 1 , None
('zomaar', 'eerder') , 1 , None
('robin', 'ruiters') , 1 , None
('schaarste', 'gaat') , 1 , None
('liegen', 'keer') , 1 , None
('staat', 'geschonden') , 1 , None
('plannen', 'dankbaar') , 1 , None
('aanuit', 'knop') , 4 , None
('youth', 'need') , 1 , None
('natuurlijk', 'overlijden') , 1 , None
('fictieve', 'angst') , 1 , None
('neonatale', 'tetanus') , 2 , None
('inleiding', 'injecteren') , 1 , None
('materie', 'creren') , 1 , None
('vergroten', 'waardoor') , 1 , 1
('hand', 'positieve') ,

('covid19', 'studie') , 2 , 1
('internet', 'komen') , 1 , None
('invaders', 'vergeleek') , 1 , None
('iedere', 'aarzelen') , 1 , None
('wonen', 'proberen') , 1 , None
('veroorzaken', 'xander') , 1 , None
('betreft', 'corona') , 1 , 3
('paar', 'brullende') , 1 , None
('contactgegevens', 'open') , 1 , None
('galienmichaelnldagen', 'geroepen') , 1 , None
('snel', 'drie') , 1 , 5
('links', 'nazi') , 1 , None
('vaccins', 'analisten') , 1 , None
('trumpsigrid', 'kaag') , 1 , None
('vier', 'bevestigde') , 1 , None
('nummer', 'indien') , 1 , None
('promoten', 'politici') , 1 , None
('betroffen', 'waarbij') , 1 , None
('beginning', 'november') , 1 , None
('lekker', 'paar') , 1 , 1
('gestart', 'richt') , 1 , None
('waddeneilanden', 'zelfs') , 1 , None
('3136', 'personen') , 1 , None
('afternoon', 'actually') , 1 , None
('periode', 'rond') , 1 , 6
('paracetamol', 'voorgeschreven') , 1 , None
('storten', 'vergelijkbaar') , 1 , None
('ziekte', 'verwierp') , 1 , None
('verdere', 'lockdowns') , 1 , 1

...

('zegt', 'nieuwe') , 3 , 22
('gebaseerd', 'httpsstichtingvaccinvrijnlwatonstewachtenstaatwaarzijndeadviezenvandegezondheidsraadopgebaseerd') , 1 , None
('beeld', 'betekent') , 1 , 2
('reported', 'illnesses') , 1 , None
('ontnemen', 'combinatie') , 1 , None
('veranderingen', 'erfelijke') , 1 , None
('anoniem', 'mogen') , 1 , 1
('voorzichtig', 'genoeg') , 1 , 1
('gemeld', 'maakt') , 2 , 1
('vanwege', 'urgenda') , 1 , 1
('neerslagtekorten', 'chemtrailactiviteiten') , 2 , None
('alleen', 'indien') , 1 , 5
('aanraken', 'resulteert') , 1 , None
('nodig', 'zichstevig') , 1 , None
('potje', 'kortom') , 1 , None
('dank', 'vaxxed') , 1 , None
('informeer', 'gezondheid') , 1 , None
('opzicht', 'hoogst') , 1 , None
('gietelink', 'praten') , 1 , None
('gechanteerd', 'turkije') , 1 , 1
('gewoon', 'bibliotheken') , 1 , None
('evenals', 'wilders') , 1 , None
('measles', 'virusthe') , 1 , None
('makkelijk', 'proberen') , 1 , None
('foutloos', 'ander') , 1 , None
('scenes', 'thousands') , 1 , None
('blo

('moment', 'toestemming') , 1 , 1
('uitkijken', 'hardnekkige') , 1 , None
('waarheid', 'wereldoorlog') , 1 , None
('behouden', 'winstgevend') , 1 , None
('europese', 'globale') , 1 , None
('nieuwe', 'wegen') , 3 , 10
('gezondheidlastig', 'begrijpen') , 1 , None
('gezond', 'personeel') , 1 , 2
('articlearno', 'wellens') , 1 , None
('wachten', 'allemaal') , 2 , 3
('notice', 'rare') , 1 , None
('vijand', 'vijand') , 1 , 1
('zetten', 'sterfte') , 1 , None
('hoog', 'kwikniveau') , 1 , None
('bron', 'levende') , 1 , None
('ontwikkelde', 'quarantaineapplicatie') , 1 , None
('verwant', 'hoofdzonde') , 1 , None
('doel', 'ahriman') , 1 , None
('researchpeilingenvvd', 'engelbart') , 2 , None
('voelt', 'leeft') , 1 , 1
('negatief', 'brengen') , 1 , 1
('maatregelen', 'versoepeling') , 1 , 2
('covid19', 'sporen') , 1 , 1
('genezen', 'combinatie') , 1 , None
('vaccinbereidheid', 'nederlandse') , 1 , None
('vaccineren1', 'sinds') , 1 , None
('media', 'vaccin') , 5 , None
('ordedienst', 'zorgen') , 1 ,

('komen', 'illegale') , 1 , 1
('veroorzaken', '2014') , 1 , None
('bleef', 'controleglas') , 1 , None
('iran', 'vertrouw') , 1 , None
('wasser', 'lage') , 1 , None
('adenovirus', 'chimpansee') , 5 , None
('httpswwwvolkskrantnlnieuwsachtergrondvvdwilonderzoeknaarlagerekinderbijslagvoorvaccinatieweigeraarsb2b43825refererhttps3a2f2fwwwgooglecom2f', 'artikel') , 1 , None
('binnenlandbonusgrapperhaushandhavingpolitiedienst', 'bart') , 1 , None
('coronavaccin', 'regels') , 1 , None
('nieuw', 'advies') , 3 , 30
('weigert', 'rest') , 1 , None
('doet', 'populair') , 1 , None
('flesje', 'kinderen') , 1 , None
('ratten', 'menselijke') , 2 , None
('stoffen', 'dagelijks') , 1 , None
('echter', 'aangegeven') , 1 , None
('tweede', 'tweede') , 4 , None
('mondkapje', 'risico') , 2 , None
('krant', 'schokkende') , 1 , None
('vandeze', 'metalen') , 1 , None
('verzin', 'spanje') , 1 , None
('getrokken', 'terugkomen') , 1 , None
('verhaal', 'info') , 1 , None
('bedragen', 'onderzoek') , 1 , None
('relatief

('vaccineren', 'volksgezondheid') , 1 , None
('kwartaal', 'volgend') , 1 , 22
('autoriteiten', 'beproefde') , 1 , None
('komen', 'media') , 2 , None
('jaar', 'vermeende') , 1 , None
('350000', 'views') , 1 , None
('straks', 'ruimte') , 1 , None
('uitgeslotenen', 'enige') , 1 , None
('httpappswhointimmunizationmonitoringglobalsummarytimeseriestsincidencemeasleshtml', 'autoren') , 1 , None
('meldt', 'nadrukkelijk') , 1 , None
('grotere', 'vertraging') , 1 , 2
('uitgeput', 'daarom') , 1 , None
('maximale', 'transparantie') , 1 , 1
('onnoemelijk', 'leed') , 2 , None
('httpsnlwikipediaorgwikibevolkingvannederland', 'httpswwwvolksgezondheidenzorginfoonderwerpziekenhuiszorgcijferscontexttrendaanbodnodetrendaantalziekenhuisbedden') , 1 , None
('weerhaan', 'staat') , 1 , None
('mensen', 'valt') , 4 , 1
('definitief', 'stuk') , 1 , None
('picture', 'this') , 1 , None
('aandoen', 'gestoord') , 1 , None
('mensen', 'afspreken') , 1 , 2
('significant', 'lower') , 1 , None
('angstflauwekul', 'heen') 

KeyboardInterrupt: 